This notebook was copied with edits from VADIM KAMAEV's notebook :
https://www.kaggle.com/code/vadimkamaev/catboost-new/notebook

In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
import lightgbm as lgb
import pickle
import sys

In [2]:

# Set max display options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Load Train Data and Labels

In [3]:
dtypes = {"session_id": 'int64',
          "index": np.int16,
          "elapsed_time": np.int32,
          "event_name": 'category',
          "name": 'category',
          "level": np.int8,
          "page": np.float16,
          "room_coor_x": np.float16,
          "room_coor_y": np.float16,
          "screen_coor_x": np.float16,
          "screen_coor_y": np.float16,
          "hover_duration": np.float32,
          "text": 'category',
          "fqid": 'category',
          "room_fqid": 'category',
          "text_fqid": 'category',
          "fullscreen": np.int8,
          "hq": np.int8,
          "music": np.int8,
          "level_group": 'category'
          }
use_col = ['session_id', 'index', 'elapsed_time', 'event_name', 'name', 'level', 'page',
           'room_coor_x', 'room_coor_y', 'hover_duration', 'text', 'fqid', 'room_fqid', 'text_fqid', 'level_group']

In [4]:
targets = pd.read_csv('data/predict-student-performance-from-game-play/train_labels.csv')
targets['session'] = targets.session_id.apply(lambda x: int(x.split('_')[0]) )
targets['q'] = targets.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )
print( targets.shape )
targets.head()

(424116, 4)


,session_id,correct,session,q
0,20090312431273200_q1,1,20090312431273200,1
1,20090312433251036_q1,0,20090312433251036,1
2,20090312455206810_q1,1,20090312455206810,1
3,20090313091715820_q1,0,20090313091715820,1
4,20090313571836404_q1,1,20090313571836404,1


In [5]:
feature_df = pd.read_csv('data/feature_sort.csv')

In [6]:
feature_df.head(50)

,nabor,tip,quest,kol_col,col1,val1,col2,val2,col3,val3,kach,kach1,rez,kach2,kach3,kach4,kach5,kach6,kach7,kach8,kach9,kach10,kach11,kach12,kach13
0,5,,18,1,text_fqid,tunic.flaghouse.entry.flag_girl.symbol_recap,0,0,0,0,107.654191,0.524945,2,0.535263,0.536886,0.551005,0.548130,0.550030,0.545479,0.543337,0.544326,0.558332,0.561411,0.563527,0.566573
1,1,,18,1,name,undefined,0,0,0,0,107.636147,0.518578,3,0.527750,0.548481,0.546731,0.550173,0.550030,0.543752,0.539850,0.546219,0.556640,0.560639,0.562642,0.566082
2,4,,18,1,room_fqid,tunic.drycleaner.frontdesk,0,0,0,0,107.632898,0.527999,1,0.526662,0.529895,0.546318,0.548125,0.547923,0.544786,0.537905,0.542566,0.559101,0.564395,0.563211,0.566013
3,1,,18,1,name,close,0,0,0,0,107.584780,0.501534,9,0.523941,0.535743,0.543156,0.547942,0.549349,0.542758,0.543017,0.547333,0.558748,0.559335,0.564256,0.566134
4,8,,18,1,text,Make sure to get some old photos for the exhib...,0,0,0,0,107.566328,0.491490,12,0.523631,0.533211,0.551282,0.547190,0.543475,0.542950,0.539692,0.543701,0.561205,0.561298,0.570363,0.565351
5,12,,18,2,room_fqid,tunic.flaghouse.entry,fqid,tunic.flaghouse,0,0,107.562176,0.489859,7,0.521717,0.534562,0.546805,0.546805,0.544852,0.547595,0.541988,0.542174,0.557936,0.563316,0.567263,0.567444
6,8,,18,1,text,Wells got in trouble for littering at the Wild...,0,0,0,0,107.554098,0.487348,4,0.524546,0.533627,0.552775,0.539839,0.549271,0.541356,0.542727,0.541125,0.558197,0.563842,0.566013,0.566085
7,8,,18,1,text,Luckily there are tons of insects around here...,0,0,0,0,107.542862,0.489793,5,0.521531,0.532538,0.545429,0.551979,0.542521,0.539872,0.533184,0.539666,0.562825,0.562028,0.563576,0.568127
8,5,,18,1,text_fqid,tunic.historicalsociety.entry.directory.closeu...,0,0,0,0,107.534672,0.492235,11,0.524700,0.528804,0.542929,0.540563,0.540654,0.537583,0.532532,0.538456,0.562826,0.567604,0.566838,0.566713
9,12,,18,2,room_fqid,tunic.kohlcenter.halloffame,fqid,tunic.library,0,0,107.528531,0.488357,6,0.518474,0.531399,0.539260,0.537508,0.553039,0.538292,0.538871,0.539944,0.559386,0.564102,0.565625,0.565918


In [7]:
feature_df.describe()

,nabor,quest,kol_col,col3,val3,kach,kach1,rez,kach2,kach3,kach4,kach5,kach6,kach7,kach8,kach9,kach10,kach11,kach12,kach13
count,16800.000000,16800.000000,16800.000000,16800.0,16800.0,16800.000000,16800.000000,16800.000000,16800.000000,16800.000000,16800.000000,16800.000000,16800.000000,16800.000000,16800.000000,16800.000000,16800.000000,16800.000000,16800.000000,16800.000000
mean,8.944643,11.366964,1.583929,0.0,0.0,9.399588,0.446107,0.101071,0.578752,0.582524,0.585337,0.577461,0.583885,0.587040,0.591146,0.593276,0.595531,0.597129,0.597197,0.597620
std,2.763497,4.872520,0.492920,0.0,0.0,11.949693,0.066583,0.946525,0.047476,0.047311,0.043569,0.040039,0.041057,0.042430,0.045725,0.047082,0.045288,0.045018,0.046098,0.045957
min,1.000000,1.000000,1.000000,0.0,0.0,7.017309,0.311173,0.000000,0.488798,0.493189,0.510003,0.494818,0.518114,0.519473,0.518711,0.517277,0.518290,0.516139,0.513632,0.517096
25%,8.000000,7.000000,1.000000,0.0,0.0,7.541696,0.413031,0.000000,0.530449,0.533412,0.541738,0.544375,0.546127,0.542024,0.541033,0.540641,0.558456,0.563486,0.564432,0.564207
50%,9.000000,12.000000,2.000000,0.0,0.0,8.121067,0.438744,0.000000,0.595211,0.607985,0.604869,0.581002,0.590432,0.595132,0.608261,0.610597,0.611555,0.611670,0.611354,0.611198
75%,11.000000,16.000000,2.000000,0.0,0.0,8.321063,0.491181,0.000000,0.611702,0.617857,0.624521,0.617290,0.623341,0.628484,0.630566,0.631159,0.630965,0.632195,0.633556,0.634131
max,12.000000,18.000000,2.000000,0.0,0.0,109.242798,0.627439,13.000000,0.672261,0.673757,0.669384,0.654957,0.659989,0.663372,0.671908,0.674755,0.674639,0.674935,0.676290,0.676780


# Feature Engineer

In [8]:
def delt_time_def(df):
    df.sort_values(by=['session_id', 'elapsed_time'], inplace=True)
    df['d_time'] = df['elapsed_time'].diff(1)
    df['d_time'].fillna(0, inplace=True)
    df['delt_time'] = df['d_time'].clip(0, 103000)
    df['delt_time_next'] = df['delt_time'].shift(-1)
    return df

In [9]:
def feature_engineer(train, kol_f):
    global kol_col, kol_col_max
    kol_col = 9
    kol_col_max = 11+kol_f*2
    col = [i for i in range(0,kol_col_max)]
    new_train = pd.DataFrame(index=train['session_id'].unique(), columns=col, dtype=np.float16)  
    new_train[10] = new_train.index # "session_id"    

    # new_train[0] = train.groupby(['session_id'])['d_time'].quantile(q=0.3)
    # new_train[1] = train.groupby(['session_id'])['d_time'].quantile(q=0.8)
    # new_train[2] = train.groupby(['session_id'])['d_time'].quantile(q=0.5)
    # new_train[3] = train.groupby(['session_id'])['d_time'].quantile(q=0.65)
    new_train[0] = train.groupby(['session_id'])['d_time'].quantile(q=0.25) # 1st Quartile
    new_train[1] = train.groupby(['session_id'])['d_time'].quantile(q=0.5)  # Median
    new_train[2] = train.groupby(['session_id'])['d_time'].quantile(q=0.75) # 3rd Quartile
    new_train[3] = train.groupby(['session_id'])['d_time'].quantile(q=0.1)  # 10th percentile
    new_train[11] = train.groupby(['session_id'])['d_time'].quantile(q=0.9)  # 90th percentile

    new_train[4] = train.groupby(['session_id'])['hover_duration'].agg('mean')
    new_train[5] = train.groupby(['session_id'])['hover_duration'].agg('std')    
    new_train[6] = new_train[10].apply(lambda x: int(str(x)[:2])).astype(np.uint8) # "year"
    new_train[7] = new_train[10].apply(lambda x: int(str(x)[2:4])+1).astype(np.uint8) # "month"
    new_train[8] = new_train[10].apply(lambda x: int(str(x)[4:6])).astype(np.uint8) # "day"
    new_train[9] = new_train[10].apply(lambda x: int(str(x)[6:8])).astype(np.uint8) + new_train[10].apply(lambda x: int(str(x)[8:10])).astype(np.uint8)/60
    new_train[10] = 0
    new_train = new_train.fillna(-1)
    
    return new_train

In [10]:
def feature_next_t(row_f, new_train, train, gran_1, gran_2, i):
    global kol_col
    kol_col +=1
    col1 = row_f['col1']
    val1 = row_f['val1']
    maska = (train[col1] == val1)
    if row_f['kol_col'] == 1:       
        new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['index'].count()          
    elif row_f['kol_col'] == 2: 
        col2 = row_f['col2']
        val2 = row_f['val2']
        maska = maska & (train[col2] == val2)        
        new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['index'].count()
    return new_train

In [11]:
def feature_next_t_otvet(row_f, new_train, train, gran_1, gran_2, i):
    global kol_col
    kol_col +=1
    col1 = row_f['col1']
    val1 = row_f['val1']
    maska = (train[col1] == val1)
    if row_f['kol_col'] == 1:      
        new_train[kol_col] = train[maska]['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska]['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska]['index'].count()          
    elif row_f['kol_col'] == 2: 
        col2 = row_f['col2']
        val2 = row_f['val2']
        maska = maska & (train[col2] == val2)        
        new_train[kol_col] = train[maska]['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska]['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska]['index'].count()
    return new_train

In [12]:
def experiment_feature_next_t_otvet(row_f, new_train, train, gran_1, gran_2, i):
    global kol_col
    kol_col +=1
    if row_f['kol_col'] == 1: 
        maska = train[row_f['col1']] == row_f['val1']
        new_train[kol_col] = train[maska]['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska]['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska]['index'].count()          
    elif row_f['kol_col'] == 2: 
        col2 = row_f['col2']
        val2 = row_f['val2']
        maska = (train[col1] == val1) & (train[col2] == val2)        
        new_train[kol_col] = train[maska]['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska]['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska]['index'].count()
    return new_train

In [13]:
def feature_quest_otvet(new_train, train, quest, kol_f):
    global kol_col
    kol_col = 9
    g1 = 0.7 
    g2 = 0.3 

    feature_q = feature_df[feature_df['quest'] == quest].copy()
    feature_q.reset_index(drop=True, inplace=True)
    
    gran1 = round(kol_f * g1)
    gran2 = round(kol_f * g2)    
    for i in range(0, kol_f):         
        row_f = feature_q.loc[i]
        new_train = feature_next_t_otvet(row_f, new_train, train, i < gran1, i <  gran2, i) 
    col = [i for i in range(0,kol_col+1)]
    return new_train[col]

In [14]:
def feature_engineer_new(new_train, train, feature_q, kol_f):
    g1 = 0.7 
    g2 = 0.3 
    gran1 = round(kol_f * g1)
    gran2 = round(kol_f * g2)    
    for i in range(0, kol_f): 
        row_f = feature_q.loc[i]       
        new_train = feature_next_t(row_f, new_train, train, i < gran1, i <  gran2, i)         
    return new_train

In [15]:
def feature_quest(new_train, train, quest, kol_f):
    global kol_col
    kol_col = 9
    feature_q = feature_df[feature_df['quest'] == quest].copy()
    feature_q.reset_index(drop=True, inplace=True)
    new_train = feature_engineer_new(new_train, train, feature_q, kol_f)
    col = [i for i in range(0,kol_col+1)]
    return new_train[col]

In [16]:
# from sklearn.linear_model import LogisticRegression
# from xgboost import XGBClassifier

# from sklearn.neural_network import MLPClassifier

# def create_model(old_train, quests, models, list_kol_f):

#     kol_quest = len(quests)
    
#     # Iterate through questions
#     for q in quests:
#         print('### quest ', q, end='')
#         new_train = feature_engineer(old_train, list_kol_f[q])
#         train_x = feature_quest(new_train, old_train, q, list_kol_f[q])
#         print(' ---- ', 'train_q.shape = ', train_x.shape)

#         # TRAIN DATA
#         train_users = train_x.index.values
#         train_y = targets.loc[targets.q == q].set_index('session').loc[train_users]
    
     
#         # XGBoost model
#         xgb_model = XGBClassifier(
#             learning_rate=0.25,
#             max_depth= 10,
#             n_estimators= 600,
#             gamma = 0.01, 
#             alpha = 0.05,  
          
#         )
#         xgb_model.fit(train_x.astype('float32'), train_y['correct'])
#         models[f'{q}_xgboost'] = xgb_model


#     print('***')

#     return models

In [18]:
def create_model(old_train, quests, models, list_kol_f):
    
    kol_quest = len(quests)
    # ITERATE THRU QUESTIONS
    for q in quests:
        print('### quest ', q, end='')
        new_train = feature_engineer(old_train, list_kol_f[q])
        train_x = feature_quest(new_train, old_train, q, list_kol_f[q])
        print (' ---- ', 'train_q.shape = ', train_x.shape)
           
        # TRAIN DATA
        train_users = train_x.index.values
        train_y = targets.loc[targets.q==q].set_index('session').loc[train_users]

        # TRAIN MODEL 

        model = CatBoostClassifier(
            n_estimators = 350,
            learning_rate= 0.045,
            depth = 6,
            l2_leaf_reg = 3,
            border_count = 128
        )
        
        model.fit(train_x.astype('float32'), train_y['correct'], verbose=False)

        # SAVE MODEL, PREDICT VALID OOF
        models[f'{q}'] = model
    print('***')
    
    return models

In [19]:
models = {}
best_threshold = 0.63

In [20]:
list_kol_f = {
    1:140,3:110,
    4:120, 5:220, 6:130, 7:110, 8:110, 9:100, 10:140, 11:120,
    14: 160, 15:160, 16:130, 17:140             
             }

In [21]:
df0_4 = pd.read_csv('data/train_0_4t.csv' ) #dtype=dtypes
kol_lvl = (df0_4 .groupby(['session_id'])['level'].agg('nunique') < 5)
list_session = kol_lvl[kol_lvl].index
df0_4  = df0_4 [~df0_4 ['session_id'].isin(list_session)]
df0_4 = delt_time_def(df0_4)

quests_0_4 = [1, 3] 
# list_kol_f = {1:140,3:110}

models = create_model(df0_4, quests_0_4, models, list_kol_f)
del df0_4

### quest  1 ----  train_q.shape =  (23562, 290)
### quest  3 ----  train_q.shape =  (23562, 230)
***


In [22]:
df5_12 = pd.read_csv('data/train_5_12t.csv') #, dtype=dtypes
kol_lvl = (df5_12.groupby(['session_id'])['level'].agg('nunique') < 8)
list_session = kol_lvl[kol_lvl].index
df5_12 = df5_12[~df5_12['session_id'].isin(list_session)]
df5_12 = delt_time_def(df5_12)
quests_5_12 = [4, 5, 6, 7, 8, 9, 10, 11] 

# list_kol_f = {4:110, 5:220, 6:120, 7:110, 8:110, 9:100, 10:140, 11:120}

models = create_model(df5_12, quests_5_12, models, list_kol_f)
del df5_12

### quest  4 ----  train_q.shape =  (23561, 250)
### quest  5 ----  train_q.shape =  (23561, 450)
### quest  6 ----  train_q.shape =  (23561, 270)
### quest  7 ----  train_q.shape =  (23561, 230)
### quest  8 ----  train_q.shape =  (23561, 230)
### quest  9 ----  train_q.shape =  (23561, 210)
### quest  10 ----  train_q.shape =  (23561, 290)
### quest  11 ----  train_q.shape =  (23561, 250)
***


In [23]:
df13_22 = pd.read_csv('data/train_13_22t.csv') #, dtype=dtypes
kol_lvl = (df13_22 .groupby(['session_id'])['level'].agg('nunique') < 10)
list_session = kol_lvl[kol_lvl].index
df13_22  = df13_22 [~df13_22 ['session_id'].isin(list_session)]
df13_22 = delt_time_def(df13_22)

quests_13_22 = [14, 15, 16, 17] 
# list_kol_f = {14: 160, 15:160, 16:105, 17:140}

models = create_model(df13_22, quests_13_22, models, list_kol_f)
del df13_22

### quest  14 ----  train_q.shape =  (22986, 330)
### quest  15 ----  train_q.shape =  (22986, 330)
### quest  16 ----  train_q.shape =  (22986, 270)
### quest  17 ----  train_q.shape =  (22986, 290)
***


In [19]:
# #Saving a Model
# import joblib

# for q in quests_0_4 + quests_5_12 + quests_13_22:
#     joblib.dump(models[str(q)+"_mlp"], f'mlp_model_{q}.joblib')

In [20]:
# for q in quests_0_4 + quests_5_12 + quests_13_22:
#     models[str(q)+"_catboost"].save_model(f'cat_model_{q}.bin')

In [21]:
# for q in quests_0_4 + quests_5_12 + quests_13_22:
#     models[str(q)+"_xgboost"].save_model(f'xgb_model_{q}.bin')

In [22]:
# for q in quests_0_4 + quests_5_12 + quests_13_22:
#     models[str(q)+"_lgb"].save_model(f'lgb_model_{q}.bin')

In [23]:
# #Model Reading
# import joblib
# dir = '/kaggle/input/my-model'
# for q in quests_0_4 + quests_5_12 + quests_13_22:
#     for i in ["_catboost","_xgboost","_mlp"] :
# #             models[f'{str(q)+i}'] =  CatBoostClassifier().load_model(dir+f'cat_model_{q}.bin')

#             if i == "_catboost" :
#                 models[f'{str(q)+i}'] = CatBoostClassifier().load_model(dir+f'/cat_model_{q}.bin')
# #             elif i == "_xgboost":
# #                 models[f'{str(q)+i}'] = XGBClassifier().load_model(dir+f'/xgb_model_{q}.bin')            
#             elif i == "_mlp" :
#                 models[f'{str(q)+i}'] = joblib.load(dir+f'/mlp_model_{q}.joblib')

In [24]:
#Saving a Model
for q in quests_0_4 + quests_5_12 + quests_13_22:
    models[f'{q}'].save_model(f'catboost/cat_model_{q}.bin')

In [ ]:
#Model Reading
# dir = '/kaggle/input/catbust/'
# for q in quests_0_4 + quests_5_12 + quests_13_22:
#     models[q] = CatBoostClassifier().load_model(dir+f'cat_model_{q}.bin')

**Infer Test Data**

In [24]:
import jo_wilder

try:
    jo_wilder.make_env.__called__ = False
    env.__called__ = False
    type(env)._state = type(type(env)._state).__dict__['INIT']
except:
    pass

env = jo_wilder.make_env()
iter_test = env.iter_test()    

In [25]:
import time

In [26]:
g_end4 = 0
g_end5 = 0

list_q = {'0-4':quests_0_4, '5-12':quests_5_12, '13-22':quests_13_22}
for (test, sam_sub) in iter_test:
    sam_sub['question'] = [int(label.split('_')[1][1:]) for label in sam_sub['session_id']]    
    grp = test.level_group.values[0]   
    sam_sub['correct'] = 1
    sam_sub.loc[sam_sub.question.isin([5, 8, 10, 13, 15]), 'correct'] = 0  
    old_train = delt_time_def(test[test.level_group == grp])
       
    for q in list_q[grp]:
        
        start4 = time.time()
        new_train = feature_engineer(old_train, list_kol_f[q])
        new_train = feature_quest_otvet(new_train, old_train, q, list_kol_f[q])
#         new_train = feature_quest(new_train, old_train, q, kol_f)
        
        end4 = time.time() - start4
        g_end4 += end4
        
        start5 = time.time()        
        
        for i in ["_catboost","_xgboost","_mlp"] :
            clf = models[f'{str(q)+i}']
            if i == "_catboost" :
                p1 = clf.predict_proba(new_train.astype('float32'))[:,1]
            elif i == "_xgboost":
                p2 = clf.predict_proba(new_train.astype('float32'))[:,1] 
            #elif i == "_lgbm":
            #    p3 = clf.predict_proba(new_train.astype('float32'))[:,1] 
            elif i == "_mlp" :
                p4 = clf.predict_proba(new_train.astype('float32').fillna(0))[:,1]

        p = 0.93*p1 + 0.03*p2 + 0.04*p4  # one can experiment with different weights and see if results improve
        
        end5 = time.time() - start5
        g_end5 += end5
             
        
        mask = sam_sub.question == q 
        x = int(p[0]>best_threshold)
        sam_sub.loc[mask,'correct'] = x      
        
        
    sam_sub = sam_sub[['session_id', 'correct']]      
    env.predict(sam_sub)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


# EDA submission.csv

In [27]:
df = pd.read_csv('submission.csv')
print( df.shape )
df.head(60)

(54, 2)


,session_id,correct
0,20090109393214576_q1,1
1,20090109393214576_q2,1
2,20090109393214576_q3,1
3,20090109393214576_q4,1
4,20090109393214576_q5,0
5,20090109393214576_q6,1
6,20090109393214576_q7,1
7,20090109393214576_q8,0
8,20090109393214576_q9,1
9,20090109393214576_q10,0
